In [2]:
import re
import requests
import pandas as pd
import os.path
from os import path,listdir,rename
from bs4 import BeautifulSoup

## Create a soup object from a sample page 

For testing and trying things out without waiting to crawl the entire site.


In [3]:
# sample_page_response = requests.get('https://thelastpsychiatrist.com/2014/05/cyberbll.html')
# sample_page = sample_page_response.content
# sample_soup_object = BeautifulSoup(sample_page, 'html.parser')


In [1]:
# print(soup)

# sample_title = sample_soup_object.select('#main h1')[0].get_text()
# sample_title_string = sample_title.get_text()
# print(sample_title)

# sample_date = sample_soup_object.select('div.dated')[0].get_text()
# print(sample_date)

# sample_entry_body = sample_soup_object.select('div.entry-body')
# print(type(sample_entry_body))
# print(sample_entry_body)

# print('\n')

# sample_entry_body_string = str(sample_entry_body[0])
# print(type(sample_entry_body_string))
# print(sample_entry_body_string)


# sample_entry_more = sample_soup_object.select('div.entry-more')
# print(sample_entry_more)


# TO-DO: comment handling
# comment_date_list = []

# sample_post_comments = sample_soup_object.select('.comment')[1]
# print(sample_post_comments.prettify())



In [4]:
url_prefix = 'https://thelastpsychiatrist.com'

archives_page = url_prefix + '/archives.html'
archives_response = requests.get(archives_page)
archives = archives_response.content

archive_soup = BeautifulSoup(archives, "html.parser")
article_links = archive_soup.select('div.archive-individual a')


## Set up your data

Here we just initialize the lists that we'll be populating in a minute. Later on we'll take these lists and create a dataframe from them.

In [5]:
# Data taken from the archives pages
slug_list = []
year_list = []
month_list = [] 

# Data taken from the individually crawled pages
title_list = []
date_list = []
entry_body_list = []
entry_more_list = []

# Comments can be handled later, if you have the inclination. 
# comments_list = []


### Populate the lists

We're going to run a series of for loops that extract the necessary information from the pages and insert them into our lists above.


In [6]:
for a in article_links:
    # Break the crawled links into month/year/slug
    link = a['href']
    slug = link[40:]
    year = link[32:36]
    month = link[37:39]
    
    # Add the cleaned up data to the appropriate list
    slug_list.append(slug)
    year_list.append(year)
    month_list.append(month)

    
print(len(slug_list))
print(len(year_list))
print(len(month_list))

# print(post_slug)
# print(post_year)
# print(post_month)


700
700
700


In [7]:
# crawl all of the individual article pages and send their data into lists
for a in article_links:

    # iterate through the list of article links that you pulled from the archives.html page
    try:
        link = a['href']
        page_response = requests.get(link)
        page_content = page_response.content

        # create a soup object out of the individually crawled page
        single_page_soup_object = BeautifulSoup(page_content,'html.parser')

        # get the content from the page and put it in the right list

        # post titles
        post_title = single_page_soup_object.select('#main h1')[0].get_text()
        title_list.append(post_title)
        
        # post dates
        post_date = single_page_soup_object.select('div.dated')[0].get_text()
        date_list.append(post_date)

        # entry bodies
        post_entry_body = single_page_soup_object.select('div.entry-body')
        post_entry_body_string = str(post_entry_body[0]) # desoupify it for the dataframe
        entry_body_list.append(post_entry_body_string)
        
        # entry mores
        post_entry_more = single_page_soup_object.select('div.entry-more')
        post_entry_more_string = str(post_entry_more[0]) # desoupify it for the dataframe
        entry_more_list.append(post_entry_more_string)
        
        # to-do: insert comments logic here
      
    
    # handling for 404s 
    except:
        title_list.append('404 Not Found')
        date_list.append('Empty Page')
        entry_body_list.append('Empty Page')
        entry_more_list.append('Empty Page')
        # to-do: insert comments logic here



### Build a dataframe from the lists

In [81]:
dataframe = pd.DataFrame(zip(slug_list,year_list,month_list,title_list,date_list,entry_body_list,entry_more_list), \
                         columns=['slug','year','month','title','date','entry_body','entry_more'])

# Find and replace the absolute URLs to make them relative
dataframe.replace('https://thelastpsychiatrist.com/images/', '../../images/', regex=True, inplace=True)
dataframe.replace('https://thelastpsychiatrist.com/', '/', regex=True, inplace=True)

# clean out the advertising code at the bottom of each page
dataframe.replace(
    '<iframe allowtransparency=\"true\"\sframeborder=[\s\S]+</div>$',
    '</div>', 
    regex=True, 
    inplace=True)


### Maintenance

In [82]:
# export the dataframe to a JSON for later, so you don't have to recrawl the site every time
dataframe.to_json('tlp_mirror.json', orient='split')

# pull the json back to pick up where you left off
#dataframe = pd.read_json('tlp_mirror.json', typ='frame', orient='split')


### Create the HTML files

Create the HTML files and insert the scraped content from the dataframe.

In [83]:
# iterate through each row in the dataframe
for row in dataframe.itertuples():
    post_slug = row[1]
    post_year = str(row[2])
    post_month = str(row[3])
    post_title = row[4]
    post_date = row[5].strip()
    post_entry_body = row[6]
    post_entry_more = row[7]
    
    # check if the /[year]/ path exists
    if not path.exists('website/' + post_year):
        os.mkdir('website/' + post_year)

    # check if the /[year]/[month]/ path exists
    if not path.exists('website/' + post_year + '/' + post_month + '/'):
        os.mkdir('website/' + post_year + '/' + post_month)
        
    
    f = open('website/' + post_year + '/' + post_month + '/' + post_slug, 'w')
    f.write("""<!DOCTYPE html>
<html lang="en">
    <head>
        <title>{title} | TLP Mirror</title>   
        <link rel="stylesheet" href="../../styles.css">
    </head>

    <body>
        <h1><a href="/">TLP Mirror</a></h1>
        <h2>{title}</h2>
        <p class="date">{date}</p>
        
        {entry_body}
        {entry_more}
        
    </body>
</html>""".format(
            title=post_title,
            date=post_date,
            entry_body=post_entry_body,
            entry_more=post_entry_more
            ))
    f.close()

    

### Recreate the Archives page


In [84]:
# create a homepage with the article archives

f = open('website/index.html', 'w')

f.write("""<!DOCTYPE html>
<html lang="en">
    <head>
        <title>TLP Mirror</title>
        <link rel="stylesheet" href="styles.css">
    </head>

    <body>
        <h1><a href="/">TLP Mirror</a></h1>
        <h2>Archives</h2>
        <ul class="archives">\n""")

for row in dataframe.itertuples():
    post_slug = row[1]
    post_year = str(row[2])
    post_month = str(row[3])
    post_title = row[4]
    post_date = row[5].strip()
    
    f.write(
        '\t\t\t <li class="archive-list-item">{post_date}: <a href=\'{post_year}/{post_month}/{post_slug}\'>{post_title}</a></li>  \n'.format( \
                                                                                post_year = post_year,
                                                                                post_month = post_month,
                                                                                post_slug = post_slug,
                                                                                post_title = post_title,
                                                                                post_date = post_date))

f.write("""
        </ul>
    </body>
</html>""")

f.close()


### Grab the images
Crawl all of the individual article pages and download any images into the /images/ directory

In [76]:
# crawl all of the individual article pages and grab the images

# check that images dir has been created
if not path.exists('website/images'):
        os.mkdir('website/images')
            
# iterate through the list of article links that you pulled from the archives.html page
for a in article_links:
    try:
        link = a['href']
        page_response = requests.get(link)
        page_content = page_response.content

        # create a soup object out of the individually crawled page
        single_page_soup_object = BeautifulSoup(page_content,'html.parser')
        
        # create a list with all the images on the page
        imgs = single_page_soup_object.findAll('img')

        # download the images from the page
        for img in imgs:
            img_url = img['src']
            img_name = img_url.split('/images/')[1]
            img_dir = 'website/images/'
            
            # do some filtering so you don't download junk
            if '/images/' in img_url and 'mt-static' not in img_url and 'rss' not in img_url and 'bookmarking' not in img_url:  

                # create a request for the image and download it
                req = requests.get(img_url, stream=True)
                req.raise_for_status()
                with open(img_dir + img_name, 'wb') as fd:
                    for chunk in req.iter_content(chunk_size=50000):
                        print('Downloading a chunk of ' + img_name + '...')
                        fd.write(chunk)
    
    # handling for 404s 
    except:
        print('Error! Probably a 404. No images here anyway. Keep going!')



Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

...

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probably a 404. No images here anyway. Keep going!
Error! Probabl

In [119]:
# Rename images with %20 in them

filenames = listdir('website/images/.')
for fname in filenames:
    if '%20' in fname:
        new_fname = fname.replace('%20', ' ')
        print(fname)
        print(new_fname)
        os.rename('website/images/' + fname, 'website/images/' + new_fname)

    
print(filenames)

this%20will%20work.jpg
this will work.jpg
usweekly-maxim%20link.jpg
usweekly-maxim link.jpg
discounting%20function.jpg
discounting function.jpg
time%20health%20issue%20cover2.jpg
time health issue cover2.jpg
zyprexa%20and%20seroquel%20pie%20chartsjpeg.JPG
zyprexa and seroquel pie chartsjpeg.JPG
diagram%20pharma.PNG
diagram pharma.PNG
obama%20rolling%20stone.JPG
obama rolling stone.JPG
macruntrient%20diet%20conclusions.png
macruntrient diet conclusions.png
commando%20scene.JPG
commando scene.JPG
junie%20b%20jones-park.JPG
junie b jones-park.JPG
hand%20drawn%20fork.JPG
hand drawn fork.JPG
harvard%20exam%201.jpg
harvard exam 1.jpg
bill%20pullman.JPG
bill pullman.JPG
abilify%20augment.JPG
abilify augment.JPG
energy%20usage.PNG
energy usage.PNG
miss%20usa%20ratings.JPG
miss usa ratings.JPG
wolf%20dad%20scary.jpg
wolf dad scary.jpg
HOW%209-11%20CHANGED%20AMERICA.png
HOW 9-11 CHANGED AMERICA.png
lorcaserin%20significant%20weight%20loss.jpg
lorcaserin significant weight loss.jpg
jobs%20program

lorcaserin%201%20year%20weight%20loss.jpg
lorcaserin 1 year weight loss.jpg
jay-z%20perfoms.JPG
jay-z perfoms.JPG
flag%202008.jpg
flag 2008.jpg
fanapt%20affinity.jpg
fanapt affinity.jpg
blockade%20aafo%20dose.JPG
blockade aafo dose.JPG
eyes%20test%20friendly.jpg
eyes test friendly.jpg
axe%20chocolate%20man.jpg
axe chocolate man.jpg
standard%20of%20care.PNG
standard of care.PNG
infrared%20spectrum.JPG
infrared spectrum.JPG
wolf%20dad.jpg
wolf dad.jpg
hillary%20adams.jpg
hillary adams.jpg
pesticide%20by%20country.jpg
pesticide by country.jpg
productivity%20and%20compensation.jpg
productivity and compensation.jpg
moot%20denial.JPG
moot denial.JPG
should%20have%20done%20more%20yoga.JPG
should have done more yoga.JPG
attachment%20parenting.jpg
attachment parenting.jpg
mr.%20baffling.JPG
mr. baffling.JPG
Coca%20Cola.jpg
Coca Cola.jpg
6%20week%20trial%20design%20fanapt.png
6 week trial design fanapt.png
millennium%20falcon%20pulverized.JPG
millennium falcon pulverized.JPG
time%202009%20influe

cover%20of%20psych.JPG
cover of psych.JPG
fanapt%20significant%20weight.jpg
fanapt significant weight.jpg
grade%20inflation.jpg
grade inflation.jpg
drinking%20coffee.JPG
drinking coffee.JPG
miss%20me%20yet.jpg
miss me yet.jpg
scott%20enjoying%20a%20nice%20iced%20tea%20on%20his%20day%20off.jpg
scott enjoying a nice iced tea on his day off.jpg
sleep%20times%20boarding%20school.JPG
sleep times boarding school.JPG
geodon%20food.JPG
geodon food.JPG
hop%20easter%20bunny%20and%20eb.JPG
hop easter bunny and eb.JPG
drive%20ryan%20gosling%20watch.JPG
drive ryan gosling watch.JPG
facebook%20dad.JPG
facebook dad.JPG
scr%20loss%20aversion.JPG
scr loss aversion.JPG
j%20crew%20ad%20crop.jpg
j crew ad crop.jpg
cnbc%20ratings%2010-8.JPG
cnbc ratings 10-8.JPG
lisa%20facebook.png
lisa facebook.png
man%20borken%20mirror.jpg
man borken mirror.jpg
up%20and%20out%20unmasked.jpg
up and out unmasked.jpg
cnbc%20and%20vix.jpg
cnbc and vix.jpg
second%20life%20woman.jpg
second life woman.jpg
monkey%20and%20sky.jpg